In [1]:
from flask import Flask, jsonify, request
app = Flask(__name__)

In [2]:
import pandas as pd
import numpy as np 

In [3]:
products=pd.read_csv('file.csv')
products.head(10)

,Unnamed: 0,CustomerID,Gender,Location,Tenure_Months,Transaction_ID,Transaction_Date,Product_SKU,Product_Description,Product_Category,...,Avg_Price,Delivery_Charges,Coupon_Status,GST,Date,Offline_Spend,Online_Spend,Month,Coupon_Code,Discount_pct
0,0,17850,M,Chicago,12,16679,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,...,153.71,6.50,Used,0.1,1/1/2019,4500,2424.5,1,ELEC10,10.0
1,1,17850,M,Chicago,12,16680,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,...,153.71,6.50,Used,0.1,1/1/2019,4500,2424.5,1,ELEC10,10.0
2,2,17850,M,Chicago,12,16696,1/1/2019,GGOENEBQ078999,Nest Cam Outdoor Security Camera - USA,Nest-USA,...,122.77,6.50,Not Used,0.1,1/1/2019,4500,2424.5,1,ELEC10,10.0
3,3,17850,M,Chicago,12,16699,1/1/2019,GGOENEBQ079099,Nest Protect Smoke + CO White Battery Alarm-USA,Nest-USA,...,81.50,6.50,Clicked,0.1,1/1/2019,4500,2424.5,1,ELEC10,10.0
4,4,17850,M,Chicago,12,16700,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,...,153.71,6.50,Clicked,0.1,1/1/2019,4500,2424.5,1,ELEC10,10.0
5,5,17850,M,Chicago,12,16701,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,...,153.71,6.50,Clicked,0.1,1/1/2019,4500,2424.5,1,ELEC10,10.0
6,6,17850,M,Chicago,12,16702,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,...,153.71,6.50,Clicked,0.1,1/1/2019,4500,2424.5,1,ELEC10,10.0
7,7,17850,M,Chicago,12,16703,1/1/2019,GGOENEBQ079099,Nest Protect Smoke + CO White Battery Alarm-USA,Nest-USA,...,81.50,6.50,Not Used,0.1,1/1/2019,4500,2424.5,1,ELEC10,10.0
8,8,17850,M,Chicago,12,16704,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,...,256.88,6.50,Used,0.1,1/1/2019,4500,2424.5,1,ELEC10,10.0
9,9,17850,M,Chicago,12,16710,1/1/2019,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,...,153.71,28.78,Clicked,0.1,1/1/2019,4500,2424.5,1,ELEC10,10.0


Preprocess Data

In [4]:
products.isnull().sum() 

Unnamed: 0               0
CustomerID               0
Gender                   0
Location                 0
Tenure_Months            0
Transaction_ID           0
Transaction_Date         0
Product_SKU              0
Product_Description      0
Product_Category         0
Quantity                 0
Avg_Price                0
Delivery_Charges         0
Coupon_Status            0
GST                      0
Date                     0
Offline_Spend            0
Online_Spend             0
Month                    0
Coupon_Code            400
Discount_pct           400
dtype: int64

In [5]:
products.columns

Index(['Unnamed: 0', 'CustomerID', 'Gender', 'Location', 'Tenure_Months',
       'Transaction_ID', 'Transaction_Date', 'Product_SKU',
       'Product_Description', 'Product_Category', 'Quantity', 'Avg_Price',
       'Delivery_Charges', 'Coupon_Status', 'GST', 'Date', 'Offline_Spend',
       'Online_Spend', 'Month', 'Coupon_Code', 'Discount_pct'],
      dtype='object')

In [6]:
df=products[['CustomerID','Gender','Product_Description','Product_Category']]
df

,CustomerID,Gender,Product_Description,Product_Category
0,17850,M,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA
1,17850,M,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA
2,17850,M,Nest Cam Outdoor Security Camera - USA,Nest-USA
3,17850,M,Nest Protect Smoke + CO White Battery Alarm-USA,Nest-USA
4,17850,M,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA
...,...,...,...,...
52919,13155,F,Gift Card - $250.00,Gift Cards
52920,18077,M,Gift Card - $25.00,Gift Cards
52921,16085,M,Google Leather Perforated Journal,Notebooks & Journals
52922,16085,M,Google Spiral Leather Journal,Notebooks & Journals


Build Engine

In [7]:
def most_purchased_products(customer_id):
    customer_data = df[df['CustomerID'] == customer_id]
    if customer_data.empty:
        return None, None
    
    # Find the most frequently purchased combination of category and gender
    most_purchased_combination = customer_data.groupby(['Product_Category', 'Gender']).size().idxmax()
    
    most_purchased_category, customer_gender = most_purchased_combination
    return most_purchased_category, customer_gender


In [8]:
def filtered_10_products(customer_id):
    most_purchased_category, customer_gender = most_purchased_products(customer_id)
    
    if most_purchased_category is None or customer_gender is None:
        return "Customer not found"
    
    # Find most purchased category with same gender
    category_gender_data = df[(df['Product_Category'] == most_purchased_category) & (df['Gender'] == customer_gender)]
    
    # Check if there are enough products in the filtered dataset
    if len(category_gender_data) < 10:
        return "Not enough products in the category and gender"
    
    random_state_value = np.random.randint(1, len(category_gender_data) + 1)
    
      # Give random products
    shuffled_data = category_gender_data.sample(frac=1, random_state=random_state_value)
    
    # Select the first 10 rows
    random_10_products = shuffled_data['Product_Description'].head(10).tolist()
    
    return random_10_products

In [9]:
def recommend_products(customer_id):
    most_purchased_category, customer_gender = most_purchased_products(customer_id)
    
    if most_purchased_category is None or customer_gender is None:
        print("Customer not found")
        return
    
    print(f"The most purchased product category for CustomerID {customer_id} is: {most_purchased_category}")
    print(f"Customer gender: {customer_gender}")
    
    top_10_products = filtered_10_products(customer_id)

    print(f"\nTop 10 products in this category and gender:")
    for i, product in enumerate(top_10_products, 1):
        print(f"{i}. {product}")

In [10]:
recommend_products(12766)

The most purchased product category for CustomerID 12766 is: Apparel
Customer gender: F

Top 10 products in this category and gender:
1. Android Matrix Tee White
2. Google Men's Short Sleeve Performance Badge Tee Charcoal
3. Google Women's Short Sleeve Hero Tee Black
4. Google Men's Vintage Badge Tee Black
5. YouTube Men's Short Sleeve Hero Tee Black
6. Android Men's Pep Rally Short Sleeve Tee Navy
7. YouTube Men's Fleece Hoodie Black
8. Google Men's Performance Hero Tee Gunmetal
9. Google Men's Vintage Badge Tee Black
10. Google Women's Tee Grey


In [11]:
#import pickle

In [12]:
import dill
import json

def save_engine_as_json(filename='recommendation_engine.json'):
    engine = {
        'most_purchased_products': most_purchased_products,
        'filtered_10_products': filtered_10_products,
        'recommend_products': recommend_products
    }

    # Serialize functions using dill to strings
    serialized_engine = {key: dill.dumps(value).decode('latin1') for key, value in engine.items()}

    with open(filename, 'w') as file:
        json.dump(serialized_engine, file, indent=4)

# Save the engine as JSON
save_engine_as_json('recommendation_engine.json')

In [13]:
# def save_engine_as_pickle(filename='recommendation_engine.pkl'):
#     engine = {
#         'most_purchased_products': most_purchased_products,
#         'filtered_10_products': filtered_10_products,
#         'recommend_products': recommend_products
#     }

#     with open(filename, 'wb') as file:
#         pickle.dump(engine, file)

# save_engine_as_pickle('recommendation_engine.pkl')

In [14]:
# pickle.load(open('recommendation_engine.pkl', 'rb'))

In [15]:
# @app.route('/recommend/<int:customer_id>', methods=['GET'])
# def recommend_api(customer_id):
#     try:
#         most_purchased_category, customer_gender = most_purchased_products(customer_id)

#         if most_purchased_category is None or customer_gender is None:
#             return jsonify({"error": "Customer not found"})

#         top_10_products = filtered_10_products(customer_id)

#         response_data = {
#             "customer_id": customer_id,
#             "most_purchased_category": most_purchased_category,
#             "customer_gender": customer_gender,
#             "top_10_products": top_10_products
#         }

#         return jsonify(response_data)

#     except Exception as e:
#         return jsonify({"error": str(e)})

# if __name__ == '__main__':
#     app.run(debug=False)

In [16]:
#recommend_products(12766)